In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
from __future__ import print_function, division, absolute_import

In [43]:
import inspect

In [44]:
import numpy as np

In [45]:
from wgomoku import GomokuTools as gt, SampleDataHelper

In [46]:
import tensorflow as tf
print(tf.__version__)
tf.test.is_gpu_available()

2.0.0-beta1


True

In [47]:
N=11

In [48]:
sdh = SampleDataHelper(N=N, representation='NxNx2', edges=True)

In [49]:
def disp(sample):
    if tf.is_tensor(sample):
        sample = sample.numpy()
    s = np.rollaxis(sample, -1, 0)
    return str(s[0]+2*s[1])

In [50]:
board = "a1a2f6g7e7"

---
Coordinates of that board's stones on a padded (here:=13x13) numpy tensor:

In [51]:
print(inspect.getsource(sdh.maybe_padded_coords))

    def maybe_padded_coords(self, board):
        """
        return stones in matrix coordinates respecting the border padding
        """
        stones = gt.string_to_stones(board)
        matrix_coords = [gt.b2m([ord(s[0])-64, s[1]], self.N) for s in stones]
        if self.edges:
            return np.add(matrix_coords, [1,1])
        else: 
            return matrix_coords



In [52]:
coords = sdh.maybe_padded_coords(board)
coords

array([[11,  1],
       [10,  1],
       [ 6,  6],
       [ 5,  7],
       [ 5,  5]])

---
An empty board's representation:

In [53]:
print(inspect.getsource(sdh.template))

    def template(self):
        """
        create a fresh empty board representation
        """
        s = np.zeros([self.N,self.N], dtype=np.int16)

        if self.rep == 'NxNx1B':
            border = 1
        elif self.edges:
            if self.rep == 'NxNx1':
                border = -1
            else:
                border = 1
        if self.edges or self.rep == 'NxNx1B':
            defensive = np.pad(s, pad_width=1, constant_values=border)
        else: 
            defensive = s
        
        if self.rep == "NxNx2":
            if self.edges:
                N=self.N+2
            else:
                N=self.N
            offensive = np.zeros([N, N], dtype=np.int16)
            template = np.stack([offensive, defensive], axis=-1)
        elif self.rep == 'NxNx1B':
            offensive = np.zeros([self.N+2, self.N+2], dtype=np.int16)
            template = np.stack([offensive, defensive], axis=-1)            
        else: 
            template = defensive

      

In [54]:
# need to change the position of the channel axis to display the board nicely
np.rollaxis(sdh.template(), 2, 0)

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 

---
Eight equivalent boards for the symmetry operations: rotate 90/180/270 and flip

In [55]:
print(inspect.getsource(sdh.rot_flip))

    def rot_flip (self, coords, quarters, flip):
        """
        coords: list of tuples of matrix coordinates
        quarters: multiples of 90 degrees to rotate
        flip: boolean: Flip or not
        """
        if self.edges:
            N = self.N+2
        else:
            N = self.N

        r,c = np.transpose(coords)
        quarters = quarters % 4
        if quarters == 0:
            res = (r,c)
        elif quarters == 1:
            res = (N-c-1, r)
        elif quarters == 2:
            res = (N-r-1,N-c-1)
        elif quarters == 3:
            res = (c, N-r-1)
        if flip:
            res = res[::-1]
        return np.transpose(res)



In [56]:
print(coords)
sdh.rot_flip(coords, quarters=1, flip=False)

[[11  1]
 [10  1]
 [ 6  6]
 [ 5  7]
 [ 5  5]]


array([[11, 11],
       [11, 10],
       [ 6,  6],
       [ 5,  5],
       [ 7,  5]])

In [57]:
print(inspect.getsource(sdh.all_symmetries))

    def all_symmetries(self, coords):
        """
        All 8 equivalent game plays
        coords: border-padded coordinates
        """
            
        return [
            self.rot_flip(coords, quarters=quarters, flip=flip) 
            for quarters in range(4) for flip in [False, True]
        ]



In [58]:
all_traj = sdh.all_symmetries(coords)
all_traj

[array([[11,  1],
        [10,  1],
        [ 6,  6],
        [ 5,  7],
        [ 5,  5]]), array([[ 1, 11],
        [ 1, 10],
        [ 6,  6],
        [ 7,  5],
        [ 5,  5]]), array([[11, 11],
        [11, 10],
        [ 6,  6],
        [ 5,  5],
        [ 7,  5]]), array([[11, 11],
        [10, 11],
        [ 6,  6],
        [ 5,  5],
        [ 5,  7]]), array([[ 1, 11],
        [ 2, 11],
        [ 6,  6],
        [ 7,  5],
        [ 7,  7]]), array([[11,  1],
        [11,  2],
        [ 6,  6],
        [ 5,  7],
        [ 7,  7]]), array([[1, 1],
        [1, 2],
        [6, 6],
        [7, 7],
        [5, 7]]), array([[1, 1],
        [2, 1],
        [6, 6],
        [7, 7],
        [7, 5]])]

---
Create samples from the trajectory using Bellmann rollout:

In [59]:
print(inspect.getsource(sdh.traj_to_samples))

    def traj_to_samples(self, traj, terminal_value, gamma):
        """
        creates samples for a given trajectory, together with the bellmann-values

        traj: trajectory, represented by their padded coordinates
        terminal_value: the given value of the last state in the trajectory
        gamma: discount factor. 
        """
        samples = []
        values = []
        value = terminal_value
        to_move_first = 1
        for t in range(len(traj)):      
            moves = traj[:t+1]            
            sample = self.create_sample(moves, to_move_first)

            samples.append(sample)
            values.append(value)
            # Actually, this is the wrong order, we'll invert before returning
            if to_move_first == 0:
                value = - value
            else:
                value = - gamma * value
            to_move_first = 1 - to_move_first

        return samples, values[::-1]



Order of the dimensions: (Batch, rows, cols, channels)

In [60]:
samples, values = sdh.traj_to_samples(all_traj[0], terminal_value=1, gamma=.9)
np.shape(samples)

(5, 13, 13, 2)

In [61]:
print(values)

[0.81, -0.81, 0.9, -0.9, 1]


In [63]:
print(disp(samples[3]), values[3])

[[2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 0 0 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 2 0 0 0 0 2]
 [2 0 0 0 0 0 1 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 0 0 0 0 0 2]
 [2 2 0 0 0 0 0 0 0 0 0 0 2]
 [2 1 0 0 0 0 0 0 0 0 0 0 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2]] -0.9


---
Now, from the string rep to all 40 board/value pairs, ready to be fed in to a neural network:

In [28]:
print(inspect.getsource(sdh.from_string_with_bellmann))

    def from_string_with_bellmann(self, board, terminal_value=None, gamma=1.0):
        """
        Creates an symmetry-augmented dataset of board positions and their values.
        Values are calculated as Bellmann rollouts backwards from terminal_value with alternating 
        signs using discount factor gamma.
        The samples will have shape [N_moves*8, N+2, N+2, 2]

        If terminal_value is 1, expect the values to be [..., gamma^2, -gamma, gamma, -1, 1]

        board: The string rep of the stones, like e.g. "j10k11i9"
        N: the size of the board
        """
        
        # stones in matrix coordinates respecting the border padding
        maybe_padded_stones = self.maybe_padded_coords(board)
        if self.cut_off:
            maybe_padded_stones = maybe_padded_stones[-self.cut_off:]

        # all equivalent representations: 4xrot, 2xflip = 8 variants 
        all_traj = self.all_symmetries(maybe_padded_stones)
        
        all_samples = []
        all_valu

In [29]:
samples, values = sdh.from_string_with_bellmann(board, gamma=.9, terminal_value=1)
print(np.shape(samples))

(40, 13, 13, 2)


In [65]:
sdh = SampleDataHelper(N=N, representation='NxNx2', edges=False, cut_off=2)
samples, values = sdh.from_string_with_bellmann(board, gamma=.9, terminal_value=1)
np.shape(samples)
values

[-0.9, 1, -0.9, 1, -0.9, 1, -0.9, 1, -0.9, 1, -0.9, 1, -0.9, 1, -0.9, 1]

---
### Variants of representation

#### 1) NxNx2, no edges

In [30]:
sdh = SampleDataHelper(N=N, representation='NxNx2', edges=False)
coords = sdh.maybe_padded_coords(board)
sample = sdh.traj_to_samples(coords, terminal_value=1, gamma=.9)[0]
coords

[array([10,  0]), array([9, 0]), array([5, 5]), array([4, 6]), array([4, 4])]

In [31]:
print(disp(sample[4]))

[[0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 1 0 0 0 0]
 [0 0 0 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 0 0 0 0]]


#### 2) NxNx1, with edges

In [32]:
sdh = SampleDataHelper(N=N, representation='NxNx1', edges=True)
coords = sdh.maybe_padded_coords(board)
sample = sdh.traj_to_samples(coords, terminal_value=1, gamma=.9)[0]
coords

array([[11,  1],
       [10,  1],
       [ 6,  6],
       [ 5,  7],
       [ 5,  5]])

---
Player (1) next to move, borders are opponents

In [33]:
sample[4]

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
       [-1,  0,  0,  0,  0, -1,  0,  1,  0,  0,  0,  0, -1],
       [-1,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0, -1],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
       [-1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
       [-1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]], dtype=int16)

#### 3) NxNx1B

In [34]:
sdh = SampleDataHelper(N=N, representation='NxNx1B')
coords = sdh.maybe_padded_coords(board)
sample = sdh.traj_to_samples(coords, terminal_value=1, gamma=.9)[0]
coords

array([[11,  1],
       [10,  1],
       [ 6,  6],
       [ 5,  7],
       [ 5,  5]])

---
Player (1) is next to move. The borders are neutral

In [35]:
print(disp(sample[4]))

[[ 2  2  2  2  2  2  2  2  2  2  2  2  2]
 [ 2  0  0  0  0  0  0  0  0  0  0  0  2]
 [ 2  0  0  0  0  0  0  0  0  0  0  0  2]
 [ 2  0  0  0  0  0  0  0  0  0  0  0  2]
 [ 2  0  0  0  0  0  0  0  0  0  0  0  2]
 [ 2  0  0  0  0 -1  0  1  0  0  0  0  2]
 [ 2  0  0  0  0  0 -1  0  0  0  0  0  2]
 [ 2  0  0  0  0  0  0  0  0  0  0  0  2]
 [ 2  0  0  0  0  0  0  0  0  0  0  0  2]
 [ 2  0  0  0  0  0  0  0  0  0  0  0  2]
 [ 2  1  0  0  0  0  0  0  0  0  0  0  2]
 [ 2 -1  0  0  0  0  0  0  0  0  0  0  2]
 [ 2  2  2  2  2  2  2  2  2  2  2  2  2]]
